In [ ]:
#import the necessary modules
import pandas as pd
import numpy as np
import random

from sklearn import linear_model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
import xgboost as xgb

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Mount Drive on Colab
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

#load the data into Colab
reddit_df = pd.read_csv('/content/drive/MyDrive/reduced_dfm_tfidf.csv')
reddit_df = reddit_df.iloc[:, 1:]
reddit_df = reddit_df.drop(['X', 'subreddit.1', 'utc_datetime_str'], axis=1)

#extract the targets and the features from the dataset
y1 = reddit_df[['left_right']]
y2 = reddit_df[['auth_lib']]
y = reddit_df[['left_right', 'auth_lib']]
X = reddit_df.iloc[:, :-2]

#estimate a linear regression model for reference
lm = linear_model.LinearRegression()

cv_results = cross_validate(lm, X, y1, cv=5, scoring='neg_mean_squared_error')
mean_mse_lm1 = sum(cv_results['test_neg_mean_squared_error'])/len(cv_results['test_neg_mean_squared_error'])

cv_results = cross_validate(lm, X, y2, cv=5, scoring=('r2', 'neg_mean_squared_error'))
mean_mse_lm2 = sum(cv_results['test_neg_mean_squared_error'])/len(cv_results['test_neg_mean_squared_error'])

print(mean_mse_lm1)
print(mean_mse_lm2)

def output_performance(model, grid):
  """Takes an initialized model and a parameter grid in the form of a dictionary as input,
  estimates the model on both axes, performs a grid search across the specified parameters
  and returns average loss for the two axes.
  """
  g_search = GridSearchCV(model, grid, cv = 5, scoring='neg_mean_squared_error', verbose = 1, refit = False)

  g_search.fit(X, y1)

  model1_mse = g_search.cv_results_['mean_test_neg_mean_squared_error']
  model1_mse = model1_mse.tolist()
  mse_ind = model1_mse.index(max(model1_mse))

  g_search.fit(X, y2)

  model2_mse = g_search.cv_results_['mean_test_neg_mean_squared_error']
  model2_mse = model2_mse.tolist()
  mse_ind = model2_mse.index(max(model2_mse))

  av_mse = (max(model2_mse) + max(model1_mse))/2

  print(av_mse)
  return av_mse

#initilialize models
ridge = linear_model.Ridge()
lasso = linear_model.Lasso()
elasticnet = linear_model.ElasticNet(random_state = 42)
tree = DecisionTreeRegressor(random_state = 42)
rf = RandomForestRegressor(random_state = 42)
gbr = GradientBoostingRegressor(random_state = 42)
xgboost = = xgb.XGBRegressor()
svr = svm.SVR()

#define parameter grids for the corresponding models
lasso_ridge_grid = {'alpha':[0.001, 0.01, 0.1, 0.2, 0.5, 0.9, 1, 5, 10, 20]}

elasticnet_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.5],
                   'l1_ratio':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 0.7, 0.9, 1],
                   "tol":[0.0001]}

tree_grid = {'max_depth': [4, 6, 8, 10, 12, 15, 20, 30, 40, 50, 70, 90, 120, 150]}

rf_grid = {'max_depth': [4, 6, 8, 10, 12],
           'n_estimators': [50, 60, 70, 80, 90, 100, 120]}

gbr_grid = {'max_depth': [4, 6, 8, 10, 12],
            'n_estimators': [50, 60, 70, 80, 90, 100, 120]}

xgb_grid = {'n_estimators': [50, 60, 70, 80, 90, 100, 120],
            'max_depth': [4, 6, 8, 10, 12],
            'eta': [0.01, 0.05, 0.1, 0.3],
            'subsample': [0.5, 0.8, 0.9, 1],
            'colsample_bytree': [0.5, 0.8, 0.9, 1]}

svr_grid = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
            'gamma' : ['scale', 'auto'],
            'C' : [0.1, 0.5, 0.8, 1, 2, 5, 10],
            'epsilon' : [0.01, 0.1, 0.3, 0.5]}

#perform grid search and evaluate performance
output_performance(ridge, lasso_ridge_grid)
output_performance(lasso, lasso_ridge_grid)
output_performance(elasticnet, elasticnet_grid)
output_performance(tree, tree_grid)
output_performance(rf, rf_grid)
output_performance(gbr, gbr_grid)
output_performance(xgboost, xgb_grid)
output_performance(svr, svr_grid)

#divide the data into train and test sets for neural network estimation
random.seed(42)
idx = random.sample(range(0, len(X) - 1), round(0.2*len(X)))

test_X, train_X = X.iloc[idx,:], X.iloc[~X.index.isin(idx), :]
test_X, train_X = test_X.values[:, :], train_X.values[:, :]

test_y1, train_y1 = y1.iloc[idx,:], y1.iloc[~y1.index.isin(idx), :]
test_y1, train_y1 = test_y1.values[:, :], train_y1.values[:, :]

test_y2, train_y2 = y2.iloc[idx,:], y2.iloc[~y2.index.isin(idx), :]
test_y2, train_y2 = test_y2.values[:, :], train_y2.values[:, :]

test_y, train_y = y.iloc[idx,:], y.iloc[~y.index.isin(idx), :]
test_y, train_y = test_y.values[:, :], train_y.values[:, :]

def create_model(rate, lr, n_y):
  """Takes a dropout rate and a learning rate as decimals and a number of outputs as an integer,
  compiles a model with the architecture outlined below, returns the model
  """
  model = keras.Sequential()

  model.add(layers.Conv1D(32, 8, activation='relu', input_shape = (train_X.shape[1], 1), name = "Convolution_layer_1"))

  model.add(layers.Dense(200, activation = "relu", name = "Dense_layer_1"))

  model.add(layers.Dropout(rate, name = "Dropout_layer_1"))

  model.add(layers.Conv1D(16, 4, activation='relu', name = "Convolution_layer_2"))

  model.add(layers.Dense(100, activation = 'relu', name = "Dense_layer_2"))

  model.add(layers.Dropout(rate, name = "Dropout_layer_2"))

  model.add(layers.Flatten())

  model.add(layers.Dense(n_y, name = "Output_layer"))

  #Define optimizer
  opt = keras.optimizers.SGD(learning_rate = lr)

  model.compile(loss = 'mean_squared_error', optimizer = opt)

  return model

def evaluate_model(train_x, train_y, test_x, test_y, rate, lr, eps, n_y):
  """Accepts train and test sets for the features and targets as dataframes; dropout rate, learning rate as decimals;
  number of epochs and the number of targets as integers; then, fits the model and returns test MSE.
  """
  #Define model
  model = create_model(rate, lr, n_y)

  model.fit(train_x, train_y, epochs = eps, validation_split = 0.1)

  #Get model predictions
  y_pred = model.predict(test_x)

  #Calculate test MSE
  mse = keras.losses.MeanSquaredError()
  test_mse = mse(test_y, y_pred).numpy()

  return test_mse

#first, try to predict axes separately
evaluate_model(train_X, train_y1, test_X, test_y1, 0.2, 0.01, 20, 1)
evaluate_model(train_X, train_y2, test_X, test_y2, 0.2, 0.01, 20, 1)

#predict axes together
evaluate_model(train_X, train_y, test_X, test_y, 0.2, 0.01, 20, 2)

#try feature selection and estimate models again
corr_coefs = [np.corrcoef(X.iloc[:, i], y2.values.ravel())[0][1] for i in range(len(X.columns))]
thr = 0.05
ind = [i for i in range(len(corr_coefs)) if corr_coefs[i] >= thr or corr_coefs[i] <= -thr]
selected_features = X.columns[ind]
X = X[selected_features]